<a href="https://colab.research.google.com/github/abhinsuresh/MLTSA_project/blob/master/MLTSA_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import pylab as pl
import numpy as np
import glob	
from google.colab import drive
import os.path
from os import path

In [0]:
def list_diff(list1, list2): 
	return (list(set(list1) - set(list2)))
 
def unique(list1): 
  
    # intilize a null list 
    unique_list = [] 
      
    # traverse for all elements 
    for x in list1: 
        # check if exists in unique_list or not 
        if x not in unique_list: 
            unique_list.append(x)
    return unique_list

In [2]:
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [4]:
cd gdrive/My\ Drive/

/content/gdrive/My Drive


In [5]:
# go to you location where stocks are stored, my stocks have proper names
# for Date and Close
cd MLTSA/MLTSA_data/new

/content/gdrive/My Drive/MLTSA/MLTSA_data/new


In [0]:
# optional to retrive the data
# a package to extract stock data from yahoo finance
pip install yfinance

In [0]:
# optional to retrive the data
import yfinance as yf

In [0]:
stockdata = pd.read_csv('https://raw.githubusercontent.com/rameyamey/MLTSAProject/master/RF2.csv')

In [0]:
stockdata['Date Employed'] = pd.to_datetime(stockdata['Date Employed'])

In [0]:
# required renamin of the symbols
stockdata.Symbol[166] = 'AMD'
stockdata.Symbol[28] = 'BRK-B'
stockdata.Symbol[92] = 'BF-B' 

In [52]:
ds_list = stockdata.Symbol.values
print(len(ds_list))
print(len(unique(ds_list.tolist())))

221
219


In [0]:
for i,comp in enumerate(unique(ds_list.tolist())):
  if not path.exists(comp + '.csv'):
    print(comp, i)
    data = yf.download(comp, start='2000-01-01', end = '2020-05-01')
    data.to_csv(comp+'.csv')

In [115]:
alltss = []
allfiles = np.sort(np.array(glob.glob("*.csv")))
print(len(unique(ds_list.tolist())))
len(allfiles)

219


219

In [0]:
ds_ls = [x+'.csv' for x in ds_list]

Remove the guys you don't like

In [0]:
ds_ls.remove('ARNC.csv')
ds_ls.remove('FTV.csv')

In [118]:
list_diff(allfiles, ds_ls)

['ARNC.csv', 'FTV.csv']

In [119]:
# updating the list after removing two companies
print(len(ds_ls))
ds_ls = unique(ds_ls)
len(ds_ls)

219


217

In [35]:
tss = []
#allfiles = np.sort(np.array(glob.glob("*.csv")))
allfiles = ds_ls
name = allfiles[0].replace(".csv","")


tss = pd.read_csv(allfiles[0])[["Date", "Close"]].rename({
        "Close":name}, axis=1)
for f in allfiles[1:]:
    name = f.replace(".csv","")  
    tss = tss.merge(pd.read_csv(f)[["Date", "Close"]].rename({
        "Close":name}, axis=1), on="Date")

# Extracting the date column
date_column = tss.loc[:,['Date']]
tss.set_index("Date", inplace=True)
tss.head()
# automatically the choose common region

,APD,AES,AVB,DAL,gm,NWL,SO,BWA,TROW,key,PCAR,SBAC,msft,ALXN,BXP,STT,pep,BLL,BAC,WMT,DIS,MHK,AKAM,BIIB,CPB,ALB,BAX,BDX,BRK-B,BBY,NUE,SEE,UPS,ADM,CBOE,LMT,MSI,NTRS,NOC,UNH,...,CRM,WFC,ALGN,AVGO,CDNS,PYPL,BEN,ADSK,A,AIV,AMAT,NVDA,NOW,SIVB,SYF,ATVI,ADBE,ROST,ALK,FB,JWN,SBUX,CNC,AEE,MMM,AOS,CVX,FTNT,NTAP,NLOK,ODFL,OKE,ORCL,COF,NKE,DHR,TFX,SWKS,CVS,XRAY
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-08-24,122.405182,11.53,165.550003,42.150002,27.799999,39.970001,43.580002,43.369999,69.650002,12.94,56.389999,114.480003,41.680000,166.850006,112.330002,69.459999,91.830002,32.400002,15.290000,63.950001,95.360001,190.330002,66.519997,278.350006,47.009998,44.340000,37.169998,138.199997,129.570007,29.27,41.439999,50.810001,95.510002,43.549999,60.369999,197.460007,60.790001,67.160004,160.020004,110.430000,...,65.169998,51.250000,54.549999,109.940002,19.000000,33.750000,39.410000,49.389999,34.680000,36.410000,15.100000,20.709999,68.000000,116.459999,33.150002,26.160000,74.269997,48.330002,73.099998,82.089996,72.110001,50.340000,30.415001,40.680000,138.580002,30.459999,72.120003,40.919998,29.870001,NaN,43.919998,32.169998,36.080002,73.529999,51.935001,63.722519,125.320000,80.510002,100.209999,51.549999
2015-11-19,129.472717,9.57,177.399994,48.509998,36.139999,43.930000,45.180000,42.380001,76.379997,13.08,51.060001,105.230003,53.939999,174.479996,124.339996,72.620003,100.930000,34.205002,17.690001,60.700001,118.709999,191.380005,59.209999,289.440002,49.470001,50.689999,37.910000,149.960007,136.630005,30.67,41.810001,45.660000,104.099998,36.430000,71.730003,224.000000,72.449997,75.400002,188.259995,110.629997,...,80.639999,55.970001,67.389999,125.209999,23.129999,36.189999,41.580002,62.389999,38.500000,36.980000,18.200001,31.120001,85.750000,127.790001,31.650000,37.009998,91.760002,46.200001,79.230003,106.260002,55.599998,61.459999,27.235001,43.700001,158.380005,39.400002,90.830002,34.419998,30.969999,NaN,43.000000,28.980000,39.020000,79.580002,62.889999,73.502655,131.029999,78.000000,92.489998,61.090000
2016-02-18,121.341354,9.70,168.360001,46.459999,29.030001,37.500000,48.869999,32.639999,69.430000,10.64,51.279999,91.570000,52.189999,143.149994,111.620003,55.580002,99.620003,33.264999,12.240000,64.120003,95.169998,163.070007,53.279999,259.049988,60.959999,54.529999,38.070000,143.179993,130.860001,30.75,40.029999,43.130001,97.459999,33.900002,61.790001,214.199997,64.779999,59.080002,192.679993,117.449997,...,61.700001,47.730000,62.880001,127.070000,20.780001,36.310001,34.660000,47.750000,37.189999,36.349998,17.170000,30.040001,54.259998,86.639999,26.620001,28.879999,80.529999,54.820000,72.320000,103.470001,52.720001,56.959999,27.959999,46.279999,156.779999,34.485001,86.730003,25.590000,23.930000,NaN,41.893333,21.190001,36.630001,65.930000,58.599998,65.708870,134.539993,62.070000,96.889999,55.599998
2016-03-04,124.690102,10.79,180.020004,48.669998,31.370001,40.980000,48.630001,35.000000,72.779999,11.55,53.419998,96.580002,52.029999,144.460007,118.580002,58.669998,100.000000,34.090000,13.540000,66.779999,98.480003,186.820007,55.570000,263.940002,61.950001,59.169998,39.459999,149.149994,137.970001,33.57,42.959999,45.680000,100.400002,37.650002,64.010002,218.720001,70.949997,65.660004,191.070007,121.800003,...,71.059998,50.110001,70.389999,146.059998,22.299999,39.040001,38.470001,56.250000,39.340000,38.540001,19.389999,32.549999,58.500000,101.660004,28.600000,31.190001,86.180000,57.419998,77.669998,108.389999,54.509998,58.700001,29.375000,47.349998,160.070007,37.070000,87.930000,28.510000,26.000000,NaN,45.406666,26.360001,37.889999,69.589996,61.259998,68.771797,146.990005,73.010002,99.279999,60.160000
2016-06-06,134.653091,11.39,170.229996,41.279999,29.990000,48.060001,50.130001,34.369999,76.410004,12.84,55.759998,102.150002,52.130001,154.990005,127.059998,62.549999,102.699997,37.150002,14.520000,71.050003,98.779999,197.210007,54.529999,289.839996,61.419998,80.959999,43.970001,169.979996,141.820007,31.33

In [37]:
tss.tail()

,APD,AES,AVB,DAL,gm,NWL,SO,BWA,TROW,key,PCAR,SBAC,msft,ALXN,BXP,STT,pep,BLL,BAC,WMT,DIS,MHK,AKAM,BIIB,CPB,ALB,BAX,BDX,BRK-B,BBY,NUE,SEE,UPS,ADM,CBOE,LMT,MSI,NTRS,NOC,UNH,...,CRM,WFC,ALGN,AVGO,CDNS,PYPL,BEN,ADSK,A,AIV,AMAT,NVDA,NOW,SIVB,SYF,ATVI,ADBE,ROST,ALK,FB,JWN,SBUX,CNC,AEE,MMM,AOS,CVX,FTNT,NTAP,NLOK,ODFL,OKE,ORCL,COF,NKE,DHR,TFX,SWKS,CVS,XRAY
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-04-24,216.779999,12.86,158.380005,22.41,21.950001,12.91,57.730000,27.080000,100.339996,10.90,67.480003,305.790009,174.550003,108.120003,90.360001,57.410000,134.360001,65.269997,22.180000,129.440002,101.190002,78.959999,102.779999,297.799988,50.750000,61.529999,92.919998,268.170013,186.520004,71.879997,37.869999,28.490000,100.180000,35.990002,96.519997,381.769989,152.800003,77.389999,339.309998,291.290009,...,153.979996,26.920000,191.970001,264.809998,80.519997,120.180000,15.92,180.350006,75.750000,35.090000,51.740002,289.589996,302.940002,184.000000,16.969999,66.650002,344.100006,85.489998,28.01,190.070007,17.840000,75.580002,70.580002,73.639999,147.000000,40.389999,87.010002,103.269997,43.060001,20.959999,141.630005,27.660000,53.009998,56.150002,88.370003,165.070007,338.390015,96.510002,62.310001,39.310001
2020-04-27,222.259995,13.28,160.750000,22.16,22.450001,13.49,58.480000,28.059999,106.500000,11.61,68.809998,308.799988,174.050003,110.860001,94.139999,61.290001,134.460007,67.570000,23.469999,128.300003,106.059998,83.739998,103.070000,303.239990,51.240002,62.250000,92.669998,270.000000,187.460007,74.570000,40.500000,28.209999,102.550003,36.540001,97.139999,378.570007,154.440002,79.910004,344.809998,293.980011,...,157.630005,28.410000,196.669998,268.450012,81.309998,120.529999,16.84,182.509995,76.250000,36.570000,51.029999,297.079987,308.600006,192.630005,17.620001,67.059998,348.500000,90.830002,27.85,187.500000,20.270000,77.739998,70.820000,73.419998,153.649994,42.529999,89.709999,105.639999,42.700001,21.280001,142.600006,28.520000,53.369999,59.189999,89.370003,169.509995,350.079987,98.379997,63.500000,41.099998
2020-04-28,223.619995,13.63,162.470001,24.34,22.180000,14.23,58.060001,28.549999,113.889999,11.83,69.160004,305.309998,169.809998,106.919998,97.070000,62.750000,136.320007,67.010002,23.889999,128.000000,106.209999,87.510002,101.930000,305.220001,50.980000,64.309998,91.000000,261.920013,187.119995,77.589996,40.490002,29.670000,96.430000,37.169998,98.500000,384.730011,145.429993,80.129997,344.529999,288.359985,...,154.460007,28.870001,203.509995,264.730011,78.470001,116.139999,17.40,177.309998,75.449997,37.209999,50.419998,291.359985,296.000000,193.250000,18.740000,63.860001,333.450012,92.540001,31.58,182.910004,20.620001,78.690002,67.169998,74.250000,157.610001,43.360001,89.910004,105.440002,42.270000,20.969999,144.490005,30.530001,53.200001,63.750000,88.800003,165.330002,340.269989,100.669998,63.240002,41.630001
2020-04-29,230.669998,14.32,165.809998,27.32,23.780001,14.33,57.369999,29.670000,116.919998,12.25,69.589996,291.470001,177.429993,108.449997,98.250000,64.459999,133.360001,66.809998,24.780001,123.599998,112.250000,93.019997,97.870003,304.709991,50.180000,64.669998,91.489998,257.850006,189.610001,80.050003,42.500000,29.650000,96.110001,38.410000,99.320000,387.910004,148.919998,82.040001,331.829987,287.649994,...,160.429993,30.000000,220.580002,276.019989,82.000000,123.580002,18.49,187.899994,78.720001,38.009998,54.340000,298.459991,321.989990,201.429993,21.049999,64.790001,349.170013,97.050003,34.00,194.190002,20.379999,76.860001,66.169998,74.620003,156.259995,44.060001,94.620003,108.519997,44.880001,21.280001,148.610001,30.920000,53.880001,69.750000,88.070000,164.970001,353.130005,106.220001,62.610001,44.200001
2020-04-30,225.580002,13.25,162.949997,25.91,22.290001,13.88,56.730000,28.570000,115.629997,11.65,69.230003,289.920013,179.210007,107.470001,97.180000,63.040001,132.289993,65.589996,24.049999,121.550003,108.150002,87.720001,97.709999,296.829987,49.980000,61.430000,88.7

In [0]:
print(tss.shape)
tcs = tss.dropna()
tcs.shape

In [0]:
tss = tcs

In [6]:
cd DF/

/content/gdrive/My Drive/MLTSA/MLTSA_data/new/DF


In [0]:
tss.to_csv('tss.csv')

In [0]:
#can skip to here
tss = pd.read_csv('tss.csv')
date_column = tss.loc[:,['Date']]
tss.set_index("Date", inplace=True)

In [11]:
print('shape of my data', tss.shape)
columns = tss.columns   # required for reconstruction

shape of my data (290, 217)


Exctracting data into arrays for preproceiing and PCA

In [12]:
X_data = tss.iloc[:, 0: len(tss.columns)].values
print("Shape of X_data is", X_data.shape)
print(type(X_data))

Shape of X_data is (290, 217)
<class 'numpy.ndarray'>


In [0]:
from sklearn.preprocessing import StandardScaler

X_std = StandardScaler().fit_transform(X_data)

Standardized dataframe

In [14]:
tss_std = pd.DataFrame(data = X_std, columns = columns)
tss_std.head()

,APD,AES,AVB,DAL,gm,NWL,SO,BWA,TROW,key,PCAR,SBAC,msft,ALXN,BXP,STT,pep,BLL,BAC,WMT,DIS,MHK,AKAM,BIIB,CPB,ALB,BAX,BDX,BRK-B,BBY,NUE,SEE,UPS,ADM,CBOE,LMT,MSI,NTRS,NOC,UNH,...,CRM,WFC,ALGN,AVGO,CDNS,PYPL,BEN,ADSK,A,AIV,AMAT,NVDA,NOW,SIVB,SYF,ATVI,ADBE,ROST,ALK,FB,JWN,SBUX,CNC,AEE,MMM,AOS,CVX,FTNT,NTAP,NLOK,ODFL,OKE,ORCL,COF,NKE,DHR,TFX,SWKS,CVS,XRAY
0,-2.192022,0.329027,-0.326746,-0.292282,0.568962,-0.447454,-1.436920,0.185019,-1.445548,-0.046419,-0.607438,-2.005810,-1.903023,1.149342,0.225080,0.407745,-2.344132,-2.001363,-0.184027,-1.934697,-1.110239,0.266851,-2.020403,1.103258,-1.922147,1.378563,-1.721295,-0.226685,-0.748882,-0.649990,0.959594,0.934481,-0.264608,0.311038,-1.679628,-1.846693,-1.619367,-0.369503,-1.770565,-0.758360,...,-0.298199,0.518861,-0.298695,-0.872587,-1.882954,-1.830917,0.592082,-0.808623,0.254679,-0.013048,-1.611796,-1.210159,-1.493270,0.697923,-0.079450,-1.709201,-1.514088,-1.239028,-0.276090,-1.193060,1.309448,-1.539248,0.180996,-1.181057,1.628594,0.974863,0.558532,-0.787092,0.632748,-0.213773,-1.286702,0.051544,-0.393344,-0.450255,-0.543221,-1.618071,-1.440967,-0.674542,-1.187794,-0.613180
1,-2.040254,0.451172,-0.275606,-0.148718,0.681973,-0.378254,-1.343161,0.271210,-1.239666,0.023027,-0.464643,-1.920915,-1.771811,1.382335,0.279552,0.467507,-2.156264,-1.909417,-0.107591,-1.822579,-1.048606,0.316428,-1.812587,1.195622,-1.949602,1.674990,-1.530459,-0.003018,-0.489045,-0.496548,1.125025,0.927942,-0.039878,0.592658,-1.474777,-1.751547,-1.430480,-0.343779,-1.671364,-0.513619,...,-0.084519,0.513393,-0.045545,-0.692045,-1.721508,-1.578301,0.737670,-0.712943,0.476596,0.058710,-1.512537,-0.965281,-1.285740,0.781938,-0.000986,-1.570494,-1.311065,-1.094057,-0.292635,-1.035281,1.368678,-1.502657,0.311704,-0.947194,1.950438,1.036086,0.689690,-0.653090,0.847776,-0.055484,-1.137509,0.097509,-0.433331,-0.346743,-0.405739,-1.496949,-1.285885,-0.587329,-1.079961,-0.463595
2,-2.034394,0.519031,-0.202986,-0.190943,0.681973,-0.386395,-1.290656,0.307675,-1.117052,0.063233,-0.449344,-1.878816,-1.727131,1.539050,0.406890,0.498906,-2.159026,-1.748902,-0.107591,-1.836437,-1.049917,0.323648,-1.782175,1.228591,-2.144077,1.670321,-1.422512,0.041438,-0.457619,-0.484045,1.137642,1.006408,-0.021063,0.614086,-1.365270,-1.819941,-1.521291,-0.309152,-1.794604,-0.399602,...,-0.032543,0.498357,0.105698,-0.670914,-1.588316,-1.571124,0.795186,-0.668826,0.582511,0.139218,-1.486756,-0.933097,-1.326122,0.739930,0.030072,-1.615690,-1.227832,-1.057813,-0.402659,-1.044864,1.381840,-1.460348,0.433600,-1.000955,1.883063,1.099494,0.708052,-0.664434,0.866474,-0.026035,-1.134592,0.127289,-0.382438,-0.339750,-0.458306,-1.392729,-1.144166,-0.536940,-0.961607,-0.494487
3,-1.972866,0.519031,-0.176905,-0.158102,0.714262,-0.435242,-1.281280,0.271210,-1.032870,0.154609,-0.364348,-1.814796,-1.677362,1.636824,0.409719,0.585002,-2.157645,-1.674099,-0.015341,-1.753294,-1.091881,0.311133,-1.713748,1.433728,-2.093742,1.756681,-1.287578,0.253296,-0.338048,-0.538603,1.199328,1.001504,0.172309,0.724285,-1.349222,-1.777746,-1.462446,-0.128098,-1.673190,-0.133562,...,0.093685,0.535264,0.131781,-0.612340,-1.580244,-1.311331,0.850905,-0.750756,0.616135,0.144468,-1.500936,-0.790834,-1.267129,0.843100,0.146134,-1.448930,-1.199435,-1.098371,-0.242173,-0.952241,1.387105,-1.454631,0.683267,-0.847735,1.927634,1.092935,0.758548,-0.575099,0.879977,0.154342,-1.041243,0.163543,-0.287922,-0.284496,-0.390913,-1.327943,-1.113087,-0.568595,-0.709120,-0.450587
4,-1.966420,0.546174,-0.141106,-0.165608,0.576137,-0.443383,-1.318784,0.107115,-1.094177,0.191160,-0.413646,-1.798096,-1.672272,1.579269,0.425991,0.629569,-2.306835,-1.649165,0.029465,-1.855333,-1.066309,0.131115,-1.715015,1.458847,-2.009089,1.583961,-1.262518,0.158133,-0.390168,-0.481772,1.103995,0.954097,0.174399,0.748773,-1.402088,-1.913985,-1.529283,0.121221,-1.807384,-0.090962,...,0.164636,0.594044,0.039984,-0.723927,-1.479340,-1.351520,0.802376,-0.763934,0.696832,0.223226,-1.458396,-0.862199,-1.224991,0.892501,0.133057,-1

In [0]:
tss_std.to_csv('tss_std.csv')

In [0]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)  # only taking two components that account for 44%

In [16]:
principalComponents = pca.fit_transform(X_std)
print(principalComponents.shape)
#principalComponents

print(pca.get_params())

(290, 2)
{'copy': True, 'iterated_power': 'auto', 'n_components': 2, 'random_state': None, 'svd_solver': 'auto', 'tol': 0.0, 'whiten': False}


In [53]:
principalDf = pd.DataFrame(data = principalComponents
             ,columns = ['pc 1'])
             #, 'pc 3', 'pc 4','pc 5', 'pc 6', 
             #            'pc 7', 'pc 8', 'pc 9', 'pc 10'])

type(principalDf)
principalDf.head()

pandas.core.frame.DataFrame

In [17]:
# optional
pca.explained_variance_ratio_*100

array([45.16892801, 30.35961882])

This process the reverts the reduced dimension into the full space.

In [0]:
approximation = pca.inverse_transform(principalComponents)

In [19]:
print(principalComponents.shape)
print(approximation.shape)

(290, 2)
(290, 217)


In [0]:
tss_std_pca = pd.DataFrame(data = approximation, columns = columns)

In [0]:
tss_std_pca.to_csv('tss_std_pca.csv')

In [0]:
tss_std_pca = pd.read_csv('tss_std_pca.csv')

PCA applied full dataframe in original structure

### We need to work on the difference :(tss_std - tss_std_pca)

I hope this is the way to do it

In [21]:
pca_removed_data = tss_std - tss_std_pca
pca_removed_data.head()

,APD,AES,AVB,DAL,gm,NWL,SO,BWA,TROW,key,PCAR,SBAC,msft,ALXN,BXP,STT,pep,BLL,BAC,WMT,DIS,MHK,AKAM,BIIB,CPB,ALB,BAX,BDX,BRK-B,BBY,NUE,SEE,UPS,ADM,CBOE,LMT,MSI,NTRS,NOC,UNH,...,CRM,WFC,ALGN,AVGO,CDNS,PYPL,BEN,ADSK,A,AIV,AMAT,NVDA,NOW,SIVB,SYF,ATVI,ADBE,ROST,ALK,FB,JWN,SBUX,CNC,AEE,MMM,AOS,CVX,FTNT,NTAP,NLOK,ODFL,OKE,ORCL,COF,NKE,DHR,TFX,SWKS,CVS,XRAY
0,-0.760704,0.859537,-0.005958,-0.462900,-0.011151,0.515033,0.178882,-0.122072,-0.021636,0.304455,0.179100,-0.551064,-0.158859,-0.027254,0.274828,1.210017,-0.908720,-0.943907,0.442868,-0.457454,-0.743368,0.007190,-0.432327,2.306351,-0.292495,1.380922,-0.283163,0.867702,0.010374,0.441602,0.321884,-0.039774,-0.033007,0.620372,-0.612066,-0.211044,-0.474839,0.126203,-0.347723,0.581513,...,0.682843,0.430086,-0.711891,-0.498602,-0.856267,-1.253070,-0.599900,0.163481,1.183921,0.118799,0.064056,0.237496,-0.165577,0.821906,-0.249031,-0.083551,0.044319,-0.181964,-0.222715,-0.080162,1.037252,-0.873171,1.106681,0.069365,0.703257,0.239516,-0.002233,0.595279,-0.191706,0.055438,0.379339,0.070439,-0.382221,-0.089991,0.724195,0.018804,-0.025811,0.729846,0.309805,-0.365828
1,-0.749053,0.842999,-0.090139,-0.432975,0.016012,0.438794,0.128868,-0.132421,0.028621,0.227822,0.181373,-0.493275,-0.106847,0.205041,0.202985,1.148221,-0.861106,-0.934156,0.366996,-0.401578,-0.814908,-0.044692,-0.278569,2.361967,-0.385509,1.610112,-0.184785,0.966070,0.113431,0.449885,0.408669,-0.093342,0.097172,0.770911,-0.554548,-0.251332,-0.413299,0.012521,-0.362120,0.725020,...,0.777624,0.306241,-0.512599,-0.450807,-0.780474,-1.088030,-0.477413,0.159825,1.293823,0.061433,0.026406,0.441195,-0.034240,0.786124,-0.286420,-0.002030,0.167250,-0.199581,-0.367542,-0.074033,1.002630,-0.961650,1.197766,0.190078,0.997433,0.255327,0.042386,0.646099,-0.024509,0.111777,0.451485,-0.011407,-0.514327,-0.132523,0.712014,0.025444,-0.011541,0.699215,0.287745,-0.352158
2,-0.778984,0.881183,-0.045199,-0.495589,0.003535,0.396897,0.143694,-0.112227,0.112559,0.238119,0.164987,-0.465741,-0.088465,0.369258,0.306106,1.151421,-0.899711,-0.796050,0.334118,-0.435657,-0.843792,-0.054980,-0.268686,2.380179,-0.602957,1.592710,-0.103658,0.979724,0.110274,0.427847,0.410547,-0.017452,0.096599,0.765535,-0.479838,-0.355958,-0.535625,0.017562,-0.516079,0.811195,...,0.800745,0.269388,-0.368864,-0.457213,-0.670145,-1.101168,-0.416535,0.178844,1.372520,0.116659,0.015417,0.456198,-0.097649,0.720651,-0.276177,-0.068590,0.225223,-0.200952,-0.502179,-0.119523,0.999804,-0.947321,1.306162,0.106801,0.930827,0.314915,0.048015,0.610052,-0.009511,0.120194,0.429019,-0.005927,-0.480932,-0.155493,0.622914,0.097390,0.094360,0.718202,0.371811,-0.410366
3,-0.781610,0.824587,-0.072808,-0.504409,0.007865,0.285534,0.086096,-0.182822,0.126779,0.271501,0.190699,-0.423305,-0.082240,0.475931,0.260762,1.185283,-0.962567,-0.760577,0.363867,-0.385296,-0.938993,-0.103855,-0.233077,2.562045,-0.590115,1.653616,-0.014902,1.135658,0.164621,0.309873,0.447447,-0.032560,0.252398,0.823737,-0.527842,-0.377736,-0.533869,0.142017,-0.448721,1.028504,...,0.874353,0.262204,-0.359937,-0.451770,-0.702470,-0.879006,-0.360272,0.051765,1.356530,0.072034,-0.063607,0.571608,-0.077820,0.777535,-0.202623,0.063876,0.211184,-0.311250,-0.390638,-0.092937,0.971684,-0.994106,1.533631,0.207478,0.972138,0.296903,0.070140,0.657153,-0.007681,0.259906,0.480247,-0.018172,-0.421351,-0.158295,0.623818,0.106120,0.061147,0.631028,0.563496,-0.419809
4,-0.764136,0.864812,-0.023764,-0.499718,-0.119942,0.290119,0.059549,-0.336157,0.078127,0.322339,0.154047,-0.407350,-0.073201,0.421324,0.290008,1.240492,-1.100724,-0.729705,0.422932,-0.485114,-0.900511,-0.272721,-0.232621,2.588034,-0.502648,1.487860,0.016230,1.050811,0.126899,0.379086,0.361826,-0.072710,0.263725,0.861059,-0.568041,-0.503931,-0.590301,0.404577,-0.574616,1.078325,...,0.955233,0.333473,-0.445141,-0.550509,-0.595196,-0.911528,-0.403493,0.046510,1.446571,0.163857,-0.010969,0.500975,-0.031027,0.839052,-0.203275,0.006609,0.323718,-0.422102,-0.422140,-0.284344,0.980762,-0.909585,1.

In [0]:
# Adding date back into the data structure
finalDf = pd.concat([date_column, pca_removed_data], axis = 1)
#finalDf.set_index("Date", inplace=True) # optional

In [24]:
# optional
finalDf.head()

,Date,APD,AES,AVB,DAL,gm,NWL,SO,BWA,TROW,key,PCAR,SBAC,msft,ALXN,BXP,STT,pep,BLL,BAC,WMT,DIS,MHK,AKAM,BIIB,CPB,ALB,BAX,BDX,BRK-B,BBY,NUE,SEE,UPS,ADM,CBOE,LMT,MSI,NTRS,NOC,...,CRM,WFC,ALGN,AVGO,CDNS,PYPL,BEN,ADSK,A,AIV,AMAT,NVDA,NOW,SIVB,SYF,ATVI,ADBE,ROST,ALK,FB,JWN,SBUX,CNC,AEE,MMM,AOS,CVX,FTNT,NTAP,NLOK,ODFL,OKE,ORCL,COF,NKE,DHR,TFX,SWKS,CVS,XRAY
0,2019-03-08,-0.760704,0.859537,-0.005958,-0.462900,-0.011151,0.515033,0.178882,-0.122072,-0.021636,0.304455,0.179100,-0.551064,-0.158859,-0.027254,0.274828,1.210017,-0.908720,-0.943907,0.442868,-0.457454,-0.743368,0.007190,-0.432327,2.306351,-0.292495,1.380922,-0.283163,0.867702,0.010374,0.441602,0.321884,-0.039774,-0.033007,0.620372,-0.612066,-0.211044,-0.474839,0.126203,-0.347723,...,0.682843,0.430086,-0.711891,-0.498602,-0.856267,-1.253070,-0.599900,0.163481,1.183921,0.118799,0.064056,0.237496,-0.165577,0.821906,-0.249031,-0.083551,0.044319,-0.181964,-0.222715,-0.080162,1.037252,-0.873171,1.106681,0.069365,0.703257,0.239516,-0.002233,0.595279,-0.191706,0.055438,0.379339,0.070439,-0.382221,-0.089991,0.724195,0.018804,-0.025811,0.729846,0.309805,-0.365828
1,2019-03-11,-0.749053,0.842999,-0.090139,-0.432975,0.016012,0.438794,0.128868,-0.132421,0.028621,0.227822,0.181373,-0.493275,-0.106847,0.205041,0.202985,1.148221,-0.861106,-0.934156,0.366996,-0.401578,-0.814908,-0.044692,-0.278569,2.361967,-0.385509,1.610112,-0.184785,0.966070,0.113431,0.449885,0.408669,-0.093342,0.097172,0.770911,-0.554548,-0.251332,-0.413299,0.012521,-0.362120,...,0.777624,0.306241,-0.512599,-0.450807,-0.780474,-1.088030,-0.477413,0.159825,1.293823,0.061433,0.026406,0.441195,-0.034240,0.786124,-0.286420,-0.002030,0.167250,-0.199581,-0.367542,-0.074033,1.002630,-0.961650,1.197766,0.190078,0.997433,0.255327,0.042386,0.646099,-0.024509,0.111777,0.451485,-0.011407,-0.514327,-0.132523,0.712014,0.025444,-0.011541,0.699215,0.287745,-0.352158
2,2019-03-12,-0.778984,0.881183,-0.045199,-0.495589,0.003535,0.396897,0.143694,-0.112227,0.112559,0.238119,0.164987,-0.465741,-0.088465,0.369258,0.306106,1.151421,-0.899711,-0.796050,0.334118,-0.435657,-0.843792,-0.054980,-0.268686,2.380179,-0.602957,1.592710,-0.103658,0.979724,0.110274,0.427847,0.410547,-0.017452,0.096599,0.765535,-0.479838,-0.355958,-0.535625,0.017562,-0.516079,...,0.800745,0.269388,-0.368864,-0.457213,-0.670145,-1.101168,-0.416535,0.178844,1.372520,0.116659,0.015417,0.456198,-0.097649,0.720651,-0.276177,-0.068590,0.225223,-0.200952,-0.502179,-0.119523,0.999804,-0.947321,1.306162,0.106801,0.930827,0.314915,0.048015,0.610052,-0.009511,0.120194,0.429019,-0.005927,-0.480932,-0.155493,0.622914,0.097390,0.094360,0.718202,0.371811,-0.410366
3,2019-03-13,-0.781610,0.824587,-0.072808,-0.504409,0.007865,0.285534,0.086096,-0.182822,0.126779,0.271501,0.190699,-0.423305,-0.082240,0.475931,0.260762,1.185283,-0.962567,-0.760577,0.363867,-0.385296,-0.938993,-0.103855,-0.233077,2.562045,-0.590115,1.653616,-0.014902,1.135658,0.164621,0.309873,0.447447,-0.032560,0.252398,0.823737,-0.527842,-0.377736,-0.533869,0.142017,-0.448721,...,0.874353,0.262204,-0.359937,-0.451770,-0.702470,-0.879006,-0.360272,0.051765,1.356530,0.072034,-0.063607,0.571608,-0.077820,0.777535,-0.202623,0.063876,0.211184,-0.311250,-0.390638,-0.092937,0.971684,-0.994106,1.533631,0.207478,0.972138,0.296903,0.070140,0.657153,-0.007681,0.259906,0.480247,-0.018172,-0.421351,-0.158295,0.623818,0.106120,0.061147,0.631028,0.563496,-0.419809
4,2019-03-14,-0.764136,0.864812,-0.023764,-0.499718,-0.119942,0.290119,0.059549,-0.336157,0.078127,0.322339,0.154047,-0.407350,-0.073201,0.421324,0.290008,1.240492,-1.100724,-0.729705,0.422932,-0.485114,-0.900511,-0.272721,-0.232621,2.588034,-0.502648,1.487860,0.016230,1.050811,0.126899,0.379086,0.361826,-0.072710,0.263725,0.861059,-0.568041,-0.503931,-0.590301,0.404577,-0.574616,...,0.955233,0.333473,-0.445141,-0.550509,-0.595196,-0.911528,-0.403493,0.046510,1.446571,0.163857,-0.010969,0.500975,-0.031027,0.839052,-0.203275,0.006609,0.323718,-0.422102,-0.422140,-0.284344,0.980762,-

In [25]:
print(type(finalDf.Date[0]))
finalDf['Date']= pd.to_datetime(finalDf['Date'])
print(type(finalDf.Date[0]))

<class 'str'>
<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [0]:
finalDf.to_csv('finalDf.csv')

In [0]:
finalDf = pd.read_csv('finalDf.csv')

In [0]:
finalDf = pd.read_csv('https://raw.githubusercontent.com/abhinsuresh/MLTSA_project/master/finalDf.csv')

# This is the dataframe on which we will do the classification, we will be adding features to this dataframe

In [160]:
# optional
stockdata.head()

,Symbol,Company,Current CEO,"Gender (1 female, 2 male)",Date Employed,Age,Education Level,Number of Children they have,sector,headquarters,headquartersMSA,pop with % less than HS,Estimate!!Total!!Total population,Estimate!!Percent below poverty level!!Population for whom poverty status is determined,"Estimate!!Percent Male!!Civilian employed population 16 years and over!!Management, business, science, and arts occupations!!Management, business, and financial occupations","Estimate!!Percent Female!!Civilian employed population 16 years and over!!Management, business, science, and arts occupations!!Management, business, and financial occupations",implicit attitudes,exp1career,exp2fam
0,APD,Air Products & Chemicals Inc,Seifi Ghasemi,2,2014-07-01,76,MS,NaN,Industrial Gases,"Allentown, Pennsylvania","Allentown-Bethlehem-Easton, PA-NJ Metro Area",7.8,834615,10.7,55.2,44.8,0.363725,4.391218,3.584493
1,AES,AES Corp,Andres Gluski,2,2011-09-01,62,PhD,NaN,Independent Power Producers & Energy Traders,"Arlington, Virginia","Washington-Arlington-Alexandria, DC-VA-MD-WV M...",9.6,6138382,8.0,52.1,47.9,0.342364,4.349400,3.430855
2,AVB,"AvalonBay Communities, Inc.",Timothy Naughton,2,2013-05-01,58,MA,NaN,Residential REITs,"Arlington, Virginia[3]","Washington-Arlington-Alexandria, DC-VA-MD-WV M...",9.6,6138382,8.0,52.1,47.9,0.342364,4.349400,3.430855
3,DAL,Delta Air Lines Inc.,Ed Bastian,2,2016-05-01,62,BS,4.0,Airlines,"Atlanta, Georgia","Atlanta-Sandy Springs-Roswell, GA Metro Area",10.7,5779463,13.1,54.1,45.9,0.344999,4.355876,3.409393
4,gm,GM,Mary Barra,1,2014-01-15,58,MBA,2.0,Data Processing & Outsourced Services,"Atlanta, Georgia","Atlanta-Sandy Springs-Roswell, GA Metro Area",10.7,5779463,13.1,54.1,45.9,0.344999,4.355876,3.409393


In [35]:
# optional
stockdata = pd.read_csv('https://raw.githubusercontent.com/rameyamey/MLTSAProject/master/RF2.csv')
stockdata.shape

(221, 19)

In [53]:
stockdata.drop_duplicates(subset ='Symbol', keep = 'last', inplace = True)
stockdata.shape

(219, 19)

In [54]:
stockdata.index[stockdata['Symbol'] == 'ANET'].tolist()

[190]

In [55]:
pwd

'/content/gdrive/My Drive/MLTSA/MLTSA_data/new/DF'

In [123]:
cd DF

/content/gdrive/My Drive/MLTSA/MLTSA_data/new/DF


In [0]:
stockdata.to_csv('stockdata.csv')

In [42]:
type(stockdata['Date Employed'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [57]:
ds_l = unique(ds_list.tolist())
ds_l.remove('ARNC')
ds_l.remove('FTV')
len(ds_l)

217

In [0]:

def slope(x):
    pars = np.polyfit(range(len(x)), x.iloc[:,0], 1)
    return pars[0]
def intercept(x):
    pars = np.polyfit(range(len(x)), x.iloc[:,0], 1)
    return pars[1]  


In [0]:
slopes = []
slopes_f = []
for comp in ds_l:
  #print(comp)
  ind = stockdata.index[stockdata['Symbol'] == comp].tolist()[0]
  # stock after appointing female ceo or male ceo
  stock = finalDf[[comp,'Date']].where(finalDf.Date >= \
                            stockdata['Date Employed'].loc[stockdata.Symbol==comp][ind])
  
  # complete stock
  stock_f = finalDf[[comp,'Date']]

  stock = stock.replace([np.inf, -np.inf], np.nan).dropna(axis=0)
  stock_f = stock_f.replace([np.inf, -np.inf], np.nan).dropna(axis=0)

  # current gender of ceo
  G = stockdata['Gender (1 female, 2 male)'].where(stockdata['Symbol'] == comp)[ind]

  if not stock.empty:
    # this implies there is some contribution from newly appointed ceo in our
    # time frame
    slopes.append([comp, slope(stock), slope(stock_f), G])
  else:
    # In this limited case, am assuming previous ceo is male
    stock = finalDf[[comp,'Date']]
    slopes.append([comp, slope(stock_f), slope(stock_f), 2.0])


In [0]:
slope_data = pd.DataFrame(slopes, columns = ['Symbol', 'slope_after', 'full_slope', 'gender_for_SA'])

In [99]:
slope_data.head()

,Symbol,slope_after,full_slope,gender_for_SA
0,APD,0.006573,0.006573,2.0
1,AES,0.000281,0.000281,2.0
2,AVB,-0.001718,-0.001718,2.0
3,DAL,-0.004694,-0.004557,2.0
4,gm,-0.007051,-0.007051,1.0


In [0]:
slope_data.to_csv('slope_data_pca_2.csv')

## Idea is to take the slope(realtive slope, compared to market trend) during the part in which CEO female CEO is appointed and add it as the main predictive feture to stockdata datframe

# What all are missing, and what all are our approximations

* We will be comparing slope calculated from different time domins, and to our frist approximation, we assume this is fine since we are working with market trend removed stock data.
* Rest of the metadata features we are adding corresponds to current geography, gender diversity or location which can be same through out common domain of time or not. 
* Like for a crude example, in the case of number of childern, we don't know exactly when the ceo had these many childern, so if we work with the slope since she was appointed as the main predictive data, she might had these childern at a later point of time, so that slope might not be good assement tool.
* My point is this "slope" might not able to used to calssify all the features which we have, we should keep this in the back of our mind; crossed fingers.
* May be we could extract the entire slope as well as a second predictive feature if that's possible/feasible.


## Random forrest calssifier
###just copied from rachel

In [0]:
rf_X = Stockdata.iloc[:,:-1].values
rf_y = Stockdata.iloc[:,-1].values

In [0]:
print("breakdown between CEOGender {} {}".format(
    (Stockdata.CEOGender == 1).sum(), (Stockdata.CEOGender == 0).sum()))

breakdown between CEOGender 24 26


In [0]:
XCEOmale = rf_X[Stockdata.CEOGender == 1]
XCEOfemale = rf_X[Stockdata.CEOGender == 0]
yCEOmale =  rf_y[Stockdata.CEOGender == 1]
yCEOfemale =  rf_y[Stockdata.CEOGender == 0]

In [0]:
import sklearn
from sklearn.model_selection import train_test_split

X_trainfemaleCEO, X_testfemaleCEO, y_trainfemaleCEO, y_testfemaleCEO = train_test_split(
     XCEOfemale, yCEOfemale, test_size=0.01, random_state=42)

In [0]:
from sklearn.ensemble import RandomForestClassifier
clfemaleCEO = RandomForestClassifier(n_estimators=100, 
                        criterion='gini', max_depth=5).fit(X_trainfemaleCEO, y_trainfemaleCEO)

In [0]:
print("Random Forest Classification\n" + 
       "score on CEO classification {:.2f}".format(
           clfemaleCEO.score(X_testfemaleCEO, y_testfemaleCEO)))

Random Forest Classification
score on CEO classification 0.00
